<a href="https://colab.research.google.com/github/RockNavamuel/youtube_project/blob/main/YouTube_API_Call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import pandas as pd
import time
import requests

In [ ]:
# Keys
API_KEY = "your_api_key"
CHANNEL_ID = "UC_q5WZtFp36adwqhKpZzxwQ"

In [ ]:
def get_statistics(video_id):
    # Gathering channel statistics
    url_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_stats = requests.get(url_stats).json()

    view_count = response_stats['items'][0]['statistics']['viewCount']
    like_count = response_stats['items'][0]['statistics']['likeCount']
    comment_count = response_stats['items'][0]['statistics']['commentCount']

    return view_count, like_count, comment_count

In [ ]:
def get_videos(df):
    pageToken = ""
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken

        response = requests.get(url).json()
        time.sleep(1)

        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                upload = video['snippet']['publishTime']
                upload_date = str(upload).split("T")[0]
                upload_time = str(upload).split("T")[1]

                view_count, like_count,comment_count = get_statistics(video_id)

                # Save data to df
                df = pd.concat([df, pd.DataFrame.from_records([{'video_id': video_id, 'video_title': video_title,
                                'upload_date': upload_date, 'upload_time': upload_time,
                                'view_count': view_count, 'like_count': like_count,
                                'comment_count': comment_count}])], ignore_index= True)
        try:
            if response['nextPageToken'] != None:
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df



In [ ]:
# Build dataframe
df = pd.DataFrame(columns=["video_id","video_title", "upload_date", "upload_time", "view_count",
                           "like_count", "comment_count"])
df = get_videos(df)

In [ ]:
df2 = df.drop_duplicates(keep = 'first')

In [ ]:
df2.to_csv('youtube_api.csv')